# Dados Bloomberg

Esse notebook tem como objetivo documentar o processo para obtencao de dados na Bloomberg

### Analise dos dados obtidos na Butia

Os tickers utilizados para obter os dados da Bloomberg vieram da juncao de Tickers fornecidos pelo Gustavo junto com alguns tickers disponibilizados no Wikipedia. Eles foram salvos no arquivo tickers.csv.

In [ ]:
# Codigo do Wikipedia

from bs4 import BeautifulSoup
import urllib.request


def get_sp500_tickers_from_wiki ():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    req = urllib.request.urlopen(url)
    article = req.read().decode()
    soup = BeautifulSoup(article, 'html.parser')
    tables = soup.find_all('table', class_='sortable')
    
    tickers_list = []
    for tr in tables[1].find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        x = [td.text.strip() for td in tds[:4]]
        if (x[3]!= ''):
            tickers_list.append(x[3])

    for tr in tables[0].find_all('tr'):
        tds = tr.find_all('td')
        if not tds:
            continue
        x = [td.text.strip() for td in tds[:4]]
        if (x[0]!= ''):
            tickers_list.append(x[0])
            
    return list(dict.fromkeys(tickers_list))

In [63]:
v_symbolsList = pd.read_csv(os.path.join('tickers.csv'))    
v_symbolsList = [x[0] for x in v_symbolsList.values.tolist()]


Desses tickers, alguns deles obteram problemas na obtencao dos dados. Sao eles: 

In [69]:
tickers = ['AJ', 'AO', 'BEA', 'BM', 'CMCS', 'CPW', 'CTV', 'DN', 'DTV', 'GN', 'IE', 'ILM', 'JB', 'JBH', 'JDS', 'JN', 'KIM', 'KRF', 'LDO', 'LIF', 'LLT', 'LRC', 'LS', 'LY', 'MH', 'NF', 'OD', 'PO', 'PYP', 'QE', 'QRV', 'QTR', 'RJ', 'RO', 'SBA', 'SBL', 'SHL', 'SIV', 'SPL', 'TLA', 'TMU', 'TTW', 'TWT', 'UD', 'UR', 'VRT', 'WA', 'XY', 'YHO', 'ZT']

As razoes de sua nao obtencao ainda devem ser investigadas.

Abaixo, foram analisadas quantas acoes por composicao do sp500 ficaram sem seus dados.

In [70]:
import os 
import datetime
import xlrd
import collections 

# Foi criado um dicionario com composicoes.
# Este dicionario possui como key o mes e ano em que a composicao foi divulgada
# E como valores, uma lista com os tickers.
# Os dados utilizados foram obtidos pelo Gustavo no terminal da Bloomberg
compositions = dict()
for file_name in  os.listdir(os.path.join('dataset', 'sp500 composition')):
    day = (file_name[14:16]) 
    month_str = (file_name[10:13]) 
    year = (file_name[17:21])
    if(month_str == 'Apr'):
        month = 4
    if(month_str == 'Oct'):
        month = 10
    if(month_str == 'Sep'):
        month = 9
    if(month_str == 'Mar'):
        month = 3   
    workbook = xlrd.open_workbook(os.path.join('dataset', 'sp500 composition', file_name), on_demand = True)
    worksheet = workbook.sheet_by_index(0)
    first_row = []
    data = []
    for row in range(1, worksheet.nrows):
        elm = {}
        data.append(worksheet.cell_value(row,0).split(' U')[0].replace('/',' '))
    key = month_str + '/' + year
    compositions[datetime.datetime(int(year), month,1)] = data #.strftime('%Y-%m-%d')

compositions = collections.OrderedDict(sorted(compositions.items()))

In [86]:
import datetime
import dateutil.relativedelta
import os 
returns, b_sp500, b_tyde = return_from_pickle(path = os.path.join('dataset',"processed", "data_input.pickle") )
      
# Este grid de data inicial e final foi iterada, salvando em uma lista partes do DF que contem as 
# acoes para cada periodo
keys = []
sp500_size = []
stocks_in_dataset = []
for i in compositions:
    keys.append(i)
    sp500_size.append(len(compositions[i]))
    stocks_in_dataset.append(len([x for x in compositions[i] if x in returns.columns ]))
    
keys = [x.strftime("%m/%Y") for x in keys]

Pelo gráfico abaixo, é possível perceber que boa parte das ações do índice ficaram sem seus dados disponibilizados. 

In [99]:
import plotly.graph_objects as go

data = [go.Bar(y=stocks_in_dataset[7:], x=keys[7:], name="Ações no dataset"), 
        go.Bar(y=sp500_size[7:], x=keys[7:], name = 'Ações no índice')]

layout = go.Layout(xaxis=dict(type='category'))
fig = go.Figure(data = data, layout = layout)
fig.update_layout(
    title='Dados faltantes',
    xaxis_title= '',
    yaxis_title = 'Quantidade de Tickers',
#     bar_mode = 'group',
    )
fig.show()

### Obtencao de dados

Para obter os tickers da Bloomberg, o seguinte codigo foi utilzado:

In [ ]:
import os 
import traceback

import pandas as pd
import pdblp

if __name__ == '__main__':
    v_symbolsList = pd.read_csv(os.path.join('tickers.csv'))
    v_symbolsList = v_symbolsList.values.tolist()
    v_successfulDownloads = []
    v_failedDownloads = []

    con = pdblp.BCon(debug=False, port=8194, timeout=5000)
    con.start()
    for i_assetSymbol in v_symbolsList:
        i_assetSymbol = i_assetSymbol[0]
        v_fileName = os.path.join('output_data', i_assetSymbol + '.csv')
        try:
            v_dataFrame = con.bdh(i_assetSymbol + ' US Equity', 'PX_LAST', '20010101', '20191215')
        except:
            print(' failed')
            v_failedDownloads.append(i_assetSymbol)
        else:
            print(' ok')
            v_dataFrame.to_csv(i_assetSymbol.replace('/','-') +'.csv')
            v_successfulDownloads.append(i_assetSymbol)

    print("Sucessful:")
    print(v_successfulDownloads)
    print("Failed:")
    print(v_failedDownloads)

Isso deve ser rodado em um computador do bloomberg. De preferencia no computador do Matheus na Butia, em que todo o processo de instalacao ja foi feito. 

Como tickers de entrada para o codigo, é recomendado utilizar:

In [95]:
tickers = []

for t in compositions:
    tickers = tickers + compositions[t]
    
tickers = list(dict.fromkeys(tickers))